In [1]:
%matplotlib inline
import nivapy3 as nivapy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona
import numpy as np
import cartopy
import matplotlib
import cartopy.crs as ccrs

plt.style.use('ggplot')

# Preliminary checking of "1000 Lakes" sampling locations

## 1. Read NVE database

NVE has previously supplied an ESRI File Geodatabase of Norwegian lakes.

In [2]:
# Get layers in GDB
gdb_path = r'../../NVEData.gdb'
layer_list = fiona.listlayers(gdb_path)
print(layer_list)

['Innsjo']


In [3]:
# Read lakes dataset
gdf = gpd.read_file(gdb_path, layer='Innsjo')
gdf.head()

,objektType,vatnLnr,navn,hoyde_moh,areal_km2,magasinNr,magasinKategori,magasinFormal_liste,vassdragsNr,elvenavnHierarki,...,arealNorge_km2,dybdekart,nedborfeltAreal_km2,dataUttaksdato,eksportType,SHAPE_Length,SHAPE_Area,UTM_Z33_E,UTM_Z33_N,geometry
0,Innsjø,165333,None,-8888,0.006160,NaN,NaN,None,025.H,KVINA,...,0.006160,None,0.010000,2017-09-27T00:00:00,NVEs nedlastningsløsning,394.219939,6159.97130,49828.230106,6.560515e+06,"(POLYGON ((49808.83000000007 6560467.3902, 498..."
1,Innsjø,56888,Juovsajavrit,438,0.217176,NaN,NaN,None,234.GBJ,IESJÅKKA/KARASJÅKKA/TANA,...,0.217176,None,5.104375,2017-09-27T00:00:00,NVEs nedlastningsløsning,4450.517347,217176.01480,864997.937388,7.764739e+06,"(POLYGON ((864712.04 7764341.240200002, 864706..."
2,Innsjø,151207,None,-8888,0.009965,NaN,NaN,None,109.5G,LITLEDALSELVA,...,0.009965,None,0.110000,2017-09-27T00:00:00,NVEs nedlastningsløsning,549.726466,9965.47710,169206.496645,6.939095e+06,(POLYGON ((169246.5300000003 6939027.540199999...
3,Innsjø,192275,None,-8888,0.010995,NaN,NaN,None,019.CF,GJØV/ARENDALSVASSDRAGET,...,0.010995,None,0.060000,2017-09-27T00:00:00,NVEs nedlastningsløsning,422.604515,10994.76345,98523.076095,6.574547e+06,(POLYGON ((98582.62999999989 6574576.120200001...
4,Innsjø,97273,None,-8888,0.008618,NaN,NaN,None,196.A4B,TAKELVA/MÅLSELVVASSDRAGET,...,0.008618,None,0.640000,2017-09-27T00:00:00,NVEs nedlastningsløsning,369.532207,8618.14750,657612.040353,7.673926e+06,(POLYGON ((657658.1200000001 7673975.150199998...


## 2a. Read stations from Excel

Atle has supplied the sampling locations from the 2019 survey.

In [4]:
# Read sampling pts
stn_path = r'../../1000_lakes_2019.xlsx'
stn_df = pd.read_excel(stn_path)
del stn_df['comment_fm'], stn_df['comment_niva']
stn_df.head()

,index,station_id,station_code,station_name,lake_name,komm_nr,komm,fylke_nr,fylke,nve_vatn_nr,utm_n,utm_e,utm_zone,lake_area_km2,altitude_m
0,525,55.0,1228-1-16,1230 HOH,1230 HOH,1228,Odda kommune,12,Hordaland,12589.0,6.628463e+06,389081.407241,32,0.073461,1233
1,72,3062.0,434-1-11,Abbortjørna,Abbortjørna,434,Engerdal kommune,4,Hedmark,35719.0,6.911424e+06,351994.000000,33,0.050000,739
2,892,3809.0,2011-1-6,Amasjavri,Amasjavri,2011,Guovdageainnu/Kautokeino komm.,20,Finmark,49720.0,7.641815e+06,572259.000000,34,0.090000,336
3,71,16132.0,432-603,Arasjøen,NaN,432,Rendalen kommune,4,Hedmark,1363.0,6.877000e+06,635600.000000,32,1.080000,690
4,748,3676.0,1718-1-15,Aunbyvatn,Aunbyvatn,1718,Leksvik kommune,17,Nord-Trøndelag,37123.0,7.060349e+06,570085.000000,32,0.080000,331


In [5]:
# Convert co-ords to WGS84
stn_df = nivapy.spatial.utm_to_wgs84_dd(stn_df, 
                                        zone='utm_zone',
                                        east='utm_e',
                                        north='utm_n')

stn_df.head()

,index,station_id,station_code,station_name,lake_name,komm_nr,komm,fylke_nr,fylke,nve_vatn_nr,utm_n,utm_e,utm_zone,lake_area_km2,altitude_m,lat,lon
0,525,55.0,1228-1-16,1230 HOH,1230 HOH,1228,Odda kommune,12,Hordaland,12589.0,6.628463e+06,389081.407241,32,0.073461,1233,59.779099,7.024378
1,72,3062.0,434-1-11,Abbortjørna,Abbortjørna,434,Engerdal kommune,4,Hedmark,35719.0,6.911424e+06,351994.000000,33,0.050000,739,62.304998,12.144987
2,892,3809.0,2011-1-6,Amasjavri,Amasjavri,2011,Guovdageainnu/Kautokeino komm.,20,Finmark,49720.0,7.641815e+06,572259.000000,34,0.090000,336,68.877993,22.796999
3,71,16132.0,432-603,Arasjøen,NaN,432,Rendalen kommune,4,Hedmark,1363.0,6.877000e+06,635600.000000,32,1.080000,690,62.001020,11.589488
4,748,3676.0,1718-1-15,Aunbyvatn,Aunbyvatn,1718,Leksvik kommune,17,Nord-Trøndelag,37123.0,7.060349e+06,570085.000000,32,0.080000,331,63.663993,10.415984


## 2b. Read stations from Aquamonitor

Roar has created a project in AM (project ID 12433) and added the new stations to it. Liv Bente and I have then corrected the co-ordinates based on my initial screening of Atle's spreadsheet. The aim of this code is to check that everything is now correct.

In [2]:
# Connect to db
eng = nivapy.da.connect()

Username:  ···
Password:  ········


Connection successful.


In [3]:
# Query projects
stn_df = nivapy.da.select_ndb_project_stations([12433],
                                               eng,
                                               drop_dups=True)

# Add UTM Z33
stn_df = nivapy.spatial.wgs84_dd_to_utm(stn_df)

print(len(stn_df))
assert stn_df['station_id'].is_unique
stn_df.head()

1005


,station_id,station_code,station_name,station_type,longitude,latitude,utm_north,utm_east,utm_zone
0,26070,221-1-2,Langtjern,Innsjø,11.850274,59.808643,6.634299e+06,323345.999797,33
1,26071,101-2-7,Hokksjøen,Innsjø,11.563586,59.004423,6.545620e+06,302651.000302,33
2,26072,402-2-13,Sætertjern,Innsjø,12.446711,60.060222,6.660863e+06,357866.000000,33
3,26073,419-1-25,Mjøgsjøen,Innsjø,11.842186,60.328578,6.692181e+06,325659.000153,33
4,26074,425-2-2,Kottern,Innsjø,12.517008,60.590729,6.719770e+06,364003.000014,33


In [4]:
# Get vatn_nrs from db
stn_ids = stn_df['station_id'].values
sql = ("SELECT station_id, value_t AS am_vatn_nr "
       "FROM nivadatabase.station_attributes "
       "WHERE attribute_id = 4 "
       "AND (1, station_id) IN (%s)" % ','.join('(1, %d)' % i for i in stn_ids))
nve_df = pd.read_sql(sql, eng)
nve_df.head()

,station_id,am_vatn_nr
0,27040,1296
1,27052,1482
2,27055,26509
3,27075,63606
4,45764,10026


In [9]:
stn_df = pd.merge(stn_df, nve_df, how='left', on='station_id')
stn_df.head()

,station_id,station_code,station_name,station_type,longitude,latitude,utm_north,utm_east,utm_zone,am_vatn_nr
0,26070,221-1-2,Langtjern,Innsjø,11.850274,59.808643,6.634299e+06,323345.999797,33,3208
1,26071,101-2-7,Hokksjøen,Innsjø,11.563586,59.004423,6.545620e+06,302651.000302,33,3608
2,26072,402-2-13,Sætertjern,Innsjø,12.446711,60.060222,6.660863e+06,357866.000000,33,4332
3,26073,419-1-25,Mjøgsjøen,Innsjø,11.842186,60.328578,6.692181e+06,325659.000153,33,4055
4,26074,425-2-2,Kottern,Innsjø,12.517008,60.590729,6.719770e+06,364003.000014,33,3794


In [10]:
# Link pts to polys
df = nivapy.spatial.identify_point_in_polygon(stn_df,
                                              gdf,
                                              pt_col='station_id',
                                              poly_col='vatnLnr',
                                              lat_col='latitude',
                                              lon_col='longitude')

# Join NVE name
df = pd.merge(df, gdf[['vatnLnr', 'navn']],
              how='left', on='vatnLnr')
df.head()

,station_id,station_code,station_name,station_type,longitude,latitude,utm_north,utm_east,utm_zone,am_vatn_nr,vatnLnr,navn
0,26070,221-1-2,Langtjern,Innsjø,11.850274,59.808643,6.634299e+06,323345.999797,33,3208,3208.0,Langtjenn
1,26071,101-2-7,Hokksjøen,Innsjø,11.563586,59.004423,6.545620e+06,302651.000302,33,3608,3608.0,Hokksjøen
2,26072,402-2-13,Sætertjern,Innsjø,12.446711,60.060222,6.660863e+06,357866.000000,33,4332,4332.0,Sætertjenn
3,26073,419-1-25,Mjøgsjøen,Innsjø,11.842186,60.328578,6.692181e+06,325659.000153,33,4055,4055.0,Mjøgsjøen
4,26074,425-2-2,Kottern,Innsjø,12.517008,60.590729,6.719770e+06,364003.000014,33,3794,3794.0,Kottern


## Which sampling locations are *not* within a lake?

In [11]:
# Which locations are not in a lake?
not_in_lake = df[pd.isna(df['vatnLnr'])]
print(len(not_in_lake), 'station locations are not within a lake.')
not_in_lake.to_csv(r'../../not_in_lake_round2.csv', encoding='utf-8')
not_in_lake

4 station locations are not within a lake.


,station_id,station_code,station_name,station_type,longitude,latitude,utm_north,utm_east,utm_zone,am_vatn_nr,vatnLnr,navn
865,26971,2021-1-2,HOH 450,Innsjø,24.664647,69.647796,7.756241e+06,873779.258600,33,NaN,NaN,NaN
906,27014,1711-3-1,Skurdalsvatnet,Innsjø,12.122000,63.372000,7.030269e+06,356116.256054,33,736,NaN,NaN
967,45671,2030-623,ST.SAMETTI,Elv,27.084386,69.488029,7.755411e+06,969921.591065,33,NaN,NaN,NaN
999,71720,807-x,Surtetjørn,Innsjø,9.125968,59.774258,6.640888e+06,170416.071590,33,NaN,NaN,NaN


## Mismatches between NVE "vatn" numbers

Atle's spreadsheet has an NVE "Vatn Nr" for each lake. The dpo not alweays match the "Vatn Nr" for the polygon enclosing the sampling point in NVE's lakes dataset.

In [16]:
# The following "vatn_nrs" do not agree
vatn_mismatch = df.dropna(subset=['vatnLnr']) 
vatn_mismatch['vatnLnr'] = vatn_mismatch['vatnLnr'].astype(int).astype(str)
vatn_mismatch = vatn_mismatch[vatn_mismatch['am_vatn_nr'] != vatn_mismatch['vatnLnr']]
vatn_mismatch.to_csv(r'../../vatn_mismatch_round2.csv', encoding='utf-8')
vatn_mismatch.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,station_id,station_code,station_name,station_type,longitude,latitude,utm_north,utm_east,utm_zone,am_vatn_nr,vatnLnr,navn


In [ ]:
## Update vatn_nrs
#with eng.begin() as conn:
#    for idx, row in df.iterrows():
#        if not pd.isna(row['vatnLnr']):
#            nve_id = str(int(row['vatnLnr']))
#            stn_id = row['station_id']
#            
#            sql = ("SELECT * FROM nivadatabase.station_attributes "
#                   "WHERE station_id = %s "
#                   "AND attribute_id = 4" % stn_id)
#            attr_df = pd.read_sql(sql, eng)
#            
#            if len(attr_df) == 1:                      
#                sql = ("UPDATE nivadatabase.station_attributes "
#                       "SET value_t = %s "
#                       "WHERE station_id = %s "
#                       "AND attribute_id = 4" % (nve_id, stn_id))
#                conn.execute(sql)   
#                
#            elif len(attr_df) == 0:
#                sql = ("INSERT INTO nivadatabase.station_attributes "
#                       "(station_id, attribute_id, value_t) "
#                       "VALUES (%s, 4, %s) " % (stn_id, nve_id))
#                conn.execute(sql)  
#            
#            else:
#                raise ValueError

In [ ]:
def chunker(df, size):
    """ Split a dataframe into chunks of length 'size'.
    """
    return (df[pos:pos + size] for pos in range(0, len(df), size))

In [ ]:
# Size of square buffer around monitoring point
buff = 500

# Loop over chunks
for df_idx, df_part in enumerate(chunker(not_in_lake.reset_index(drop=True), 20)):
    fig = plt.figure(figsize=(15,20))

    # Loop over stations
    for idx, row in df_part.reset_index(drop=True).iterrows():    
        zone = row['utm_zone']
        north = row['utm_north']
        east = row['utm_east']

        # Define co-ord system
        crs = ccrs.UTM(zone)  

        # Plot
        ax = fig.add_subplot(5, 4, idx+1, projection=crs)
        ax.set_extent([east - buff, east + buff, north - buff, north + buff], crs=crs)
        
        ax.add_wms(wms='https://openwms.statkart.no/skwms1/wms.topo4',
                   layers=['topo4_WMS'])

        ax.scatter(east, 
                   north, 
                   s=50,
                   c='r', 
                   edgecolors='k', 
                   transform=crs)
   
        ax.set_title('%s (%s)' % (row['station_name'], row['station_code']), fontsize=12)
     
    # Save 
    out_png = r'../../grid_maps/not_in_lake_round2/grid_plot_%03d.png' % (df_idx + 1)
    plt.savefig(out_png, dpi=150)
    plt.clf()
    plt.close()

In [ ]:
# Size of square buffer around monitoring point
buff = 1000

# Loop over chunks
for df_idx, df_part in enumerate(chunker(vatn_mismatch.reset_index(drop=True), 20)):
    fig = plt.figure(figsize=(15,20))

    # Loop over stations
    for idx, row in df_part.reset_index(drop=True).iterrows():    
        zone = row['utm_zone']
        north = row['utm_n']
        east = row['utm_e']

        # Define co-ord system
        crs = ccrs.UTM(zone)  

        # Plot
        ax = fig.add_subplot(5, 4, idx+1, projection=crs)
        ax.set_extent([east - buff, east + buff, north - buff, north + buff], crs=crs)
        
        ax.add_wms(wms='https://openwms.statkart.no/skwms1/wms.topo4',
                   layers=['topo4_WMS'])

        ax.scatter(east, 
                   north, 
                   s=50,
                   c='r', 
                   edgecolors='k', 
                   transform=crs)

        ax.set_title('%s (%s)' % (row['station_name'], row['station_code']), fontsize=12)
     
    # Save 
    out_png = r'../../grid_maps/vatn_nr_mismatch/grid_plot_%03d.png' % (df_idx + 1)
    plt.savefig(out_png, dpi=150)
    plt.clf()
    plt.close()

In [ ]:
df_agg = df.groupby('komm_nr')

# Size of square buffer around monitoring point
buff = 1000

# Loop over kommune
for komm_nr, grp_df in df_agg:
    # Loop over chunks
    for df_idx, df_part in enumerate(chunker(grp_df.reset_index(drop=True), 20)):
        fig = plt.figure(figsize=(15,20))

        # Loop over stations
        for idx, row in df_part.reset_index(drop=True).iterrows():    
            zone = row['utm_zone']
            north = row['utm_n']
            east = row['utm_e']

            # Define co-ord system
            crs = ccrs.UTM(zone)  

            # Plot
            ax = fig.add_subplot(5, 4, idx+1, projection=crs)
            ax.set_extent([east - buff, east + buff, north - buff, north + buff], crs=crs)

            ax.add_wms(wms='https://openwms.statkart.no/skwms1/wms.topo4',
                       layers=['topo4_WMS'])

            ax.scatter(east, 
                       north, 
                       s=50,
                       c='r', 
                       edgecolors='k', 
                       transform=crs)

            ax.set_title('%s (%s)' % (row['station_name'], row['station_code']), fontsize=12)

        # Save 
        out_png = r'../../grid_maps/all_sites/grid_plot_komm%04d_plot%03d.png' % (komm_nr, df_idx + 1)
        plt.savefig(out_png, dpi=150)
        plt.clf()
        plt.close()